In [1]:
import pandas as pd 

In [12]:
from civic_scraper.platforms import CivicPlusSite
url = 'http://ny-wyomingcounty.civicplus.com/AgendaCenter'
# url = 'http://oh-circleville.civicplus.com/AgendaCenter'
site = CivicPlusSite(url)

In [ ]:
assets_metadata = site.scrape(start_date='2020-01-01', end_date='2022-01-01')

# Scrape Legislation Only

## legistar without videos available

'https://phila.legistar.com/Legislation.aspx'

'https://columbus.legistar.com/Legislation.aspx'

'https://phoenix.legistar.com/Legislation.aspx'

## legistar with videos available

'https://seattle.legistar.com/Legislation.aspx'

'https://newark.legistar.com/Legislation.aspx'

'https://durhamcounty.legistar.com/Legislation.aspx'

'https://alexandria.legistar.com/Legislation.aspx'

'https://fortworthgov.legistar.com/Legislation.aspx'

## potentially new sources

https://denver.legistar.com/Calendar.aspx

https://jaxcityc.legistar.com/Calendar.aspx

In [ ]:
from civic_scraper.platforms import LegistarSite
import pytz

In [ ]:
timezone = 'US/Pacific' 
site = LegistarSite('https://durhamcounty.legistar.com/Calendar.aspx', timezone=timezone)
# site = LegistarSite('https://fortworthgov.legistar.com/Calendar.aspx',) 
# r = site.scrape(start_date='2021-02-01', end_date='2021-03-01')

In [ ]:
from urllib.parse import parse_qs, urlparse
from dateutil.parser import parse
from legistar.bills import LegistarBillScraper

In [51]:
webscraper = LegistarBillScraper()

In [138]:
url = 'https://durhamcounty.legistar.com/Calendar.aspx'

In [65]:
webscraper.BASE_URL = urlparse(url).netloc
webscraper.LEGISLATION_URL = url
webscraper.TIMEZONE = timezone
webscraper.date_format = "%m/%d/%Y %I:%M %p"

In [66]:
parse('2021-01-01')

datetime.datetime(2021, 1, 1, 0, 0)

In [70]:
pages = webscraper.searchLegislation(
    created_after=parse('2021-01-01'), created_before=parse('2021-06-01')
)

/Users/spangher/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'durhamcounty.legistar.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/spangher/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'durhamcounty.legistar.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
pages_list = list(pages)

In [85]:
type(pages_list[3])

lxml.html.HtmlElement

In [91]:
from lxml import etree
import pandas as pd 

In [93]:
t = pd.read_html(etree.tostring(pages_list[0], pretty_print=True).decode())

/var/folders/xh/qnyq7yzj0r328_7hnb7pgxth0000gp/T/ipykernel_18720/248126114.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  t = pd.read_html(etree.tostring(pages_list[0], pretty_print=True).decode())


In [ ]:
# required to instantiate webscraper
ac = AssetCollection()
start_year = int(start_date[:4])
events = [event[0] for event in webscraper.events(since=start_year)]
for event in events:
    meeting_meta = self._extract_meeting_meta(event, webscraper)
    for asset_type in asset_list:
        # Skip if a dictionary containing 'url' key is not present for the given asset type
        try:
            asset = self._create_asset(event, meeting_meta, asset_type)
        except TypeError:
            continue
        # Apply date and other filters
        if self._skippable(
            asset, start_date, end_date, file_size=file_size, download=download
        ):
            continue
        ac.append(asset)
if download:
    asset_dir = Path(self.cache.path, "assets")
    asset_dir.mkdir(parents=True, exist_ok=True)
    for asset in ac:
        if asset.url:
            dir_str = str(asset_dir)
            asset.download(target_dir=dir_str, session=webscraper)
return ac

# Manually Scrape

In [1]:
import playwright
from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright
from IPython.display import display, HTML
import unidecode
from bs4 import BeautifulSoup
import re 
import time
import html
import unicodedata
import pandas as pd 

def process_row(td, col_name, row_chunk=None):
    col_name = col_name.replace(':', '')
    text = td.get_text().strip()
    text = unicodedata.normalize("NFKD", text)
    if row_chunk is None:
        row_chunk = {}
        
    row_chunk[col_name] = text
    a = td.find('a')
    if (a) and (a.attrs.get('href') != None):
        row_chunk[f"{col_name}_href"] = a.attrs.get('href')
    return row_chunk

In [2]:
urls = [
    # 'https://fortworthgov.legistar.com/Calendar.aspx',
    'https://cityofdallas.legistar.com/Calendar.aspx',
    'https://seattle.legistar.com/Calendar.aspx',
    'https://alexandria.legistar.com/Calendar.aspx',
    'https://phila.legistar.com/Calendar.aspx',
    'https://columbus.legistar.com/Calendar.aspx',
    'https://newark.legistar.com/Calendar.aspx',
    'https://phoenix.legistar.com/Calendar.aspx',
    # 
    'https://durhamcounty.legistar.com/Calendar.aspx',
    'https://denver.legistar.com/Calendar.aspx',
    'https://fortworthgov.legistar.com/Calendar.aspx',
    'https://jaxcityc.legistar.com/Calendar.aspx',
]

In [3]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()

In [4]:
# go to the page 
url = urls[-1]
await page.goto(url)

# change to "all years"
await page.click('#ctl00_ContentPlaceHolder1_lstYears_Input')
await page.click('#ctl00_ContentPlaceHolder1_lstYears_DropDown > div > ul > li:nth-child(1)')

In [6]:
html = await page.content()
soup = BeautifulSoup(html)
page_bar = soup.find(attrs={'class': 'rgWrap rgInfoPart'}).get_text().strip()
total_num_pages = int(re.search(r'Page \d of (\d+)', page_bar)[1])

In [7]:
total_num_pages

16

### Get all meetings from meeting tables

In [11]:
# click through all the meetings
pager_sel_path = '''#ctl00_ContentPlaceHolder1_gridCalendar_ctl00 > 
            thead > 
            tr.rgPager > 
            td > table > tbody > tr > td > div.rgWrap.rgNumPart > a'''

all_rows = []
# button_count = await page.locator(button_count_sel).count()
for i in range(1, total_num_pages + 1):
    print(f'running page {i}')
    num_pages_sel = f'''{pager_sel_path}:has-text("{i}")'''
    await page.wait_for_selector(pager_sel_path)
    on_page = await page.is_visible(num_pages_sel)
    if on_page:
        await page.locator(num_pages_sel).first.click()
    else:
        # if i == 14:
            # break
        print(f'didn\'t find locator at: {i}')
        await page.locator(f'''{pager_sel_path}[title="Next Pages"]''').first.click()
    time.sleep(5)

    ## parse html 
    html = await page.content()
    soup = BeautifulSoup(html)

    tables = soup.find_all('table', attrs={'class': 'rgMasterTable'})
    for table in tables:
        table_head = table.find('thead')
        table_head = list(filter(lambda x: (x != '\n') and (x.attrs.get('class', [None])[0] != 'rgPager'), list(table_head.children)))[0]
        table_body = table.find_all('tbody')
    
        table_col_names = list(map(lambda x: x.get_text().strip(), table_head.find_all('th')))
        rows = table.find_all('tr', attrs={'class': 'rgRow'}) + table.find_all('tr', attrs={'class': 'rgAltRow'})
        print(f'num rows: {len(rows)}')
        for row in rows:
            td_cols = row.find_all('td')
            row_chunk = {}
            for td_name, td in zip(table_col_names, td_cols):
                row_chunk = process_row(td, td_name, row_chunk)
            all_rows.append(row_chunk)

running page 1
num rows: 9
num rows: 100
running page 2
num rows: 9
num rows: 100
running page 3
num rows: 9
num rows: 100
running page 4
num rows: 9
num rows: 100
running page 5
num rows: 9
num rows: 100
running page 6
num rows: 9
num rows: 100
running page 7
num rows: 9
num rows: 100
running page 8
num rows: 9
num rows: 100
running page 9
num rows: 9
num rows: 100
running page 10
num rows: 9
num rows: 100
running page 11
didn't find locator at: 11
num rows: 9
num rows: 100
running page 12
num rows: 9
num rows: 100
running page 13
num rows: 9
num rows: 100
running page 14
num rows: 9
num rows: 100
running page 15
num rows: 9
num rows: 100
running page 16
num rows: 9
num rows: 39


In [12]:
all_rows_df = pd.DataFrame(sorted(all_rows, key=lambda x: -len(x)))

In [17]:
all_rows_df.shape

(1683, 21)

In [22]:
all_rows_df['Meeting Date'] = pd.to_datetime(all_rows_df['Meeting Date'])

In [28]:
import datetime
all_rows_df.loc[lambda df: df['Meeting Date'] < datetime.datetime.now()].head(2)

,Name,Name_href,Meeting Date,Add to Calendar,Add to Calendar_href,Meeting Time,Meeting Location,Meeting Details,Meeting Details_href,Agenda,...,Minutes,Minutes_href,Addendum,Addendum_href,Proposed Amends & Subs,Proposed Amends & Subs_href,Video,Video_href,,_href
0,City Council,DepartmentDetail.aspx?ID=39526&GUID=AF102B62-F...,2024-05-14,Add to Calendar,View.ashx?M=IC&ID=1110896&GUID=44CDCE20-00E8-4...,5:00 PM,"Council Chamber\n1st Floor, City Hall",Meeting details,MeetingDetail.aspx?ID=1110896&GUID=44CDCE20-00...,Agenda,...,Minutes,View.ashx?M=M&ID=1110896&GUID=44CDCE20-00E8-4F...,Addendum,View.ashx?M=E2&ID=1110896&GUID=44CDCE20-00E8-4...,Proposed Amends & Subs,View.ashx?M=E3&ID=1110896&GUID=44CDCE20-00E8-4...,Video,#,NaN,NaN
1,City Council,DepartmentDetail.aspx?ID=39526&GUID=AF102B62-F...,2024-03-26,Add to Calendar,View.ashx?M=IC&ID=1110893&GUID=A10CF4BE-28CC-4...,5:00 PM,"Council Chamber\n1st Floor, City Hall",Meeting details,MeetingDetail.aspx?ID=1110893&GUID=A10CF4BE-28...,Agenda,...,Minutes,View.ashx?M=M&ID=1110893&GUID=A10CF4BE-28CC-46...,Addendum,View.ashx?M=E2&ID=1110893&GUID=A10CF4BE-28CC-4...,Proposed Amends & Subs,View.ashx?M=E3&ID=1110893&GUID=A10CF4BE-28CC-4...,Video,#,NaN,NaN


In [35]:
all_rows_df = all_rows_df.drop_duplicates().loc[lambda df: df['Video_href'].notnull()]

In [13]:
# all_rows_df.to_csv('../data/alexandria-meeting-dates.csv')
# all_rows_df.to_csv('../data/seattle-meeting-dates.csv')
# all_rows_df.to_csv('../data/dallas-meeting-dates.csv')
# all_rows_df.to_csv('../data/philadelphia-meeting-dates.csv')
# all_rows_df.to_csv('../data/columbus-meeting-dates.csv')
# all_rows_df.to_csv('../data/newark-meeting-dates.csv')
# all_rows_df.to_csv('../data/phoenix-meeting-dates.csv')
all_rows_df.to_csv('../data/durham-meeting-dates.csv')

In [74]:
# all_rows_df = pd.read_csv('../data/durham-meeting-dates.csv', index_col=0)

In [51]:
all_rows_df = all_rows_df.loc[lambda df: ~df['Name'].str.contains('(inactive)')] #.loc[lambda df: df['Name'] == 'City Council'].head(3)

/var/folders/xh/qnyq7yzj0r328_7hnb7pgxth0000gp/T/ipykernel_79289/3584300320.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_rows_df = all_rows_df.loc[lambda df: ~df['Name'].str.contains('(inactive)')] #.loc[lambda df: df['Name'] == 'City Council'].head(3)


### Get all legislation from meetings

In [52]:
from urllib.parse import urlparse

In [ ]:
meeting_detail_hrefs = ['Meeting Details_href', 'Details_href']
meeting_href = None
for c in meeting_detail_hrefs:
    if c in all_rows_df.columns:
        meeting_href = c

meeting_details = (
    all_rows_df
        # .loc[lambda df: df['Name'] == 'City Council']
        [meeting_href]
        .dropna()
)
url_parts = urlparse(url)
domain = url_parts[0] + '://' + url_parts[1]

In [19]:
from tqdm.auto import tqdm
tqdm.pandas()

In [92]:
rows_for_all_meetings = []
for meeting_page in tqdm(meeting_details):
    await page.goto(domain + '/' + meeting_page)
    await page.wait_for_selector('#ctl00_ContentPlaceHolder1_gridMain_ctl00')
    time.sleep(.1)
    ## parse table 
    html = await page.content()
    soup = BeautifulSoup(html)
    table = soup.find('table', attrs={'class': 'rgMasterTable'})
    t_head = table.find('thead')
    t_body = table.find('tbody')
    col_names = list(map(lambda x: x.get_text(), t_head.find_all('th')))
    col_names = list(map(lambda x: unicodedata.normalize("NFKD", x).replace(':', ''), col_names))
    rows = t_body.find_all('tr')
    all_meeting_rows = []
    for tr in rows:
        row_chunk = {}
        for td_name, td in zip(col_names, tr.find_all('td')):
            row_chunk = process_row(td, td_name, row_chunk)
        all_meeting_rows.append(row_chunk)
    all_meeting_rows_df = pd.DataFrame(all_meeting_rows)

    # get video link
    page_promise = context.wait_for_event('page', timeout=2000)
    await page.locator('#ctl00_ContentPlaceHolder1_hypVideo').click()
    try:
        new_page = await page_promise
        await new_page.wait_for_load_state()
        video_url = new_page.url
        await new_page.close()
    except PlaywrightTimeoutError as t:
        print(f'timeout error: {domain}/{meeting_page}')
        video_url = None

    all_meeting_rows_df['key'] = meeting_page
    all_meeting_rows_df['video_url'] = video_url
    ## append
    rows_for_all_meetings.append(all_meeting_rows_df)

  0%|          | 0/381 [00:00<?, ?it/s]

Future exception was never retrieved
future: <Future finished exception=Exception('Connection closed while reading from the driver')>
Exception: Connection closed while reading from the driver
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeout error
timeou

In [94]:
final_all_rows_meeting_df = pd.concat(rows_for_all_meetings)

In [95]:
final_all_rows_meeting_df.shape

(6471, 14)

In [97]:
final_all_rows_meeting_df['video_url'].isnull().value_counts()

video_url
False    5830
True      641
Name: count, dtype: int64

In [208]:
final_all_rows_meeting_df.shape#.merge()

(69143, 13)

In [282]:
url

'https://newark.legistar.com/Calendar.aspx'

In [283]:
col_set = ['Name', 'Name_href', 'Date', 'Time', 'Location', meeting_href]
col_set = ['Name', 'Date', 'Time', meeting_href]
col_set = ['Name', 'Meeting Date', 'Meeting Time', 'Meeting Location', meeting_href, 'video_url']

(final_all_rows_meeting_df
     .rename(columns={'Name' : 'Notes'})
     .merge(all_rows_df[col_set], left_on='key', right_on=meeting_href)
     .drop(meeting_href, axis=1)
     .loc[lambda df: df['Title'].notnull()]
     .reset_index(drop=True)
     .to_csv('../data/newark/newark-meeting-info.csv')
)

In [285]:
ls ../data/*

../data/alexandria:
alexandria-meeting-dates.csv  alexandria-meeting-info.csv

../data/columbus:
columbus-meeting-dates.csv  columbus-meeting-info.csv

../data/dallas:
dallas-meeting-dates.csv  dallas-meeting-info.csv

../data/newark:
newark-meeting-dates.csv  newark-meeting-info.csv

../data/philadelphia:
inquirer-urls-to-get.csv
philadelphia-meeting-dates.csv
philadelphia-meeting-info.csv
philly-inquirer-cc-articles-to-fetch.txt.gz

../data/phoenix:
phoenix-meeting-dates.csv  phoenix-meeting-info.csv

../data/seattle:
seattle-meeting-dates.csv  seattle-meeting-info.csv
